In [1]:
import pandas as pd
import numpy as np
import os
import ast

import pyvista as pv
import panel as pn
pn.extension('vtk')

from tqdm import tqdm
from pathlib import Path

In [2]:
model = 'geneva_200m'
params_file = 'swirl_03'
output_folder = r'../Outputs'

# Import catalogues

In [3]:
level0_data = pd.read_csv(os.path.join(output_folder, f'{model}_{params_file}_day1_lvl0.csv'))
level1_data = pd.read_csv(os.path.join(output_folder, f'{model}_{params_file}_day1_lvl1.csv'))
level2a_data = pd.read_csv(os.path.join(output_folder, f'{model}_{params_file}_day1_lvl2a.csv'))

# Select eddy to track

In [4]:
id_lvl2a = 0
ids_lvl1 = ast.literal_eval(level2a_data.iloc[id_lvl2a]['id_lvl1'])

In [5]:
lvl1_eddy = level1_data[level1_data['id'].isin(ids_lvl1)]

In [6]:
parsed_ids_lvl0 = []
for item in lvl1_eddy['id_lvl0']:
    parsed_ids_lvl0.extend(ast.literal_eval(item)) 

In [7]:
lvl0_eddy = level0_data[level0_data['id'].isin(parsed_ids_lvl0)]

# Display eddy

In [26]:
def create_grid(time_index):
    eddy_cells = []
    for _, row in lvl0_eddy[lvl0_eddy['time_index'] == time_index].iterrows():
        i = np.fromstring(row['i_eddy_cells'].strip("[]"), sep=' ')
        j = np.fromstring(row['j_eddy_cells'].strip("[]"), sep=' ')
        k = np.ones(len(i)) * row['depth_index']
        
        coords = np.stack((i, j, k), axis=1)
        eddy_cells.append(coords)
        
    eddy_cells_array = np.vstack(eddy_cells)  
    
    min_i, max_i = min(eddy_cells_array[:,0]), max(eddy_cells_array[:,0])
    min_j, max_j = min(eddy_cells_array[:,1]), max(eddy_cells_array[:,1])
    min_k, max_k = min(eddy_cells_array[:,2]), max(eddy_cells_array[:,2])
    
    dims = [int(max_i - min_i)+1, int(max_j-min_j)+1, int(max_k-min_k)+1]
    
    data_display = np.zeros(dims)
    for cell in eddy_cells_array:
        data_display[int(cell[0]-min_i), int(cell[1]-min_j), int(cell[2]-min_k)] = 1

    dims = np.array(data_display.shape)
    grid = pv.ImageData(dimensions=dims, spacing=(1, 1, 1), origin=(0, 0, 0))
    grid.point_data['eddies'] = data_display.flatten(order="F")
    
    return grid

In [ ]:
def update_plot(time_index, plotter):
    plotter.clear()
    grid = create_grid(time_index)
    #plotter.add_volume(grid, scalars='eddies', opacity=[0,0.2],opacity_unit_distance=1, shade=True, cmap='viridis')
    
    surface = grid.contour(isosurfaces=[0.5], scalars='eddies')
    plotter.add_mesh(surface, color='turquoise', opacity=0.5, show_edges=True)    
    plotter.add_text(f"Time Index: {time_index}", position='upper_left', font_size=12, color='black')

    plotter.add_axes()
    plotter.camera_position = 'yz'
    plot_pane.object = plotter.ren_win

# Make video (frames)

In [46]:
frames_dir = Path(f"..//Plots//video_frames_{model}_{params_file}")
frames_dir.mkdir(exist_ok=True)

# Parameters
time_indices = np.unique(lvl0_eddy['time_index'])
frame_paths = []

# Loop through time steps and save each frame
plotter = pv.Plotter(off_screen=True)
for t in tqdm(time_indices, desc="Rendering frames"):
    update_plot(t, plotter)    
    # Save screenshot
    filename = frames_dir / f"frame_{int(t):04d}.png"
    plotter.screenshot(str(filename))
    frame_paths.append(filename)

Rendering frames:   0%|          | 0/24 [00:00<?, ?it/s]libEGL warning: failed to open /dev/dri/renderD128: Permission denied

libEGL warning: failed to open /dev/dri/renderD128: Permission denied

2025-07-22 14:31:21.524 (11585.182s) [        93487740] vtkEGLRenderWindow.cxx:351   WARN| vtkEGLRenderWindow (0x5557fd1f5600): EGL device index: 0 could not be initialized. Trying other devices...
Rendering frames: 100%|██████████| 24/24 [00:04<00:00,  5.04it/s]


# Display in dashboard

In [36]:
# Setup PyVista plotter
plotter = pv.Plotter(off_screen=True)
plot_pane = pn.pane.VTK(plotter.ren_win, sizing_mode='stretch_both')

# Create Panel slider
slider = pn.widgets.IntSlider(name="Time", start=0, end=len(np.unique(lvl0_eddy['time_index'])), step=1)
slider.param.watch(lambda e: update_plot(e.new, plotter), 'value')

# Initial plot
update_plot(0, plotter)

# Display layout
dashboard = pn.Column(slider, plot_pane)
dashboard.show()

Launching server at http://localhost:42965
